In [4]:

import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')


# fields names
ped_level= 'ped_level'
import os
general_path = os.path.dirname(os.path.abspath(os.getcwd()))

from sklearn.model_selection import train_test_split

from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, explained_variance_score,median_absolute_error
from sklearn.tree import _tree

In [5]:
ml_df = pd.read_csv('output/model_data_final.csv').drop(columns=['Unnamed: 0'])
data_feature = ml_df.drop(columns=[ped_level,'geometry','node_end','node_start'])
x_tree = data_feature.to_numpy()
y_tree = ml_df[ped_level].to_numpy()
ml_df

,day part,season,day,buildings,businesses,educationa,Health_ser,Leisure_am,Playground,Sport_faci,...,bike_trail,parks,density,SEleve1_10,closeness,betweennes,node_start,node_end,geometry,ped_level
0,0,2,0,16.036667,0.0,8,1,0,7,3,...,0,12,3.3,7,0.014734,0.004886,0,1,"LINESTRING (184322.70500000002 668574.483, 184...",1
1,1,2,0,16.036667,0.0,8,1,0,7,3,...,0,12,3.3,7,0.014734,0.004886,0,1,"LINESTRING (184322.70500000002 668574.483, 184...",0
2,2,2,0,16.036667,0.0,8,1,0,7,3,...,0,12,3.3,7,0.014734,0.004886,0,1,"LINESTRING (184322.70500000002 668574.483, 184...",2
3,3,2,0,16.036667,0.0,8,1,0,7,3,...,0,12,3.3,7,0.014734,0.004886,0,1,"LINESTRING (184322.70500000002 668574.483, 184...",0
4,0,2,1,16.036667,0.0,8,1,0,7,3,...,0,12,3.3,7,0.014734,0.004886,0,1,"LINESTRING (184322.70500000002 668574.483, 184...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473795,3,0,0,19.660000,0.0,9,3,4,2,2,...,0,6,22.9,6,0.024888,0.001354,6432,6433,"LINESTRING (178804.17200000002 665057.617, 178...",4
473796,0,0,1,19.660000,0.0,9,3,4,2,2,...,0,6,22.9,6,0.024888,0.001354,6432,6433,"LINESTRING (178804.17200000002 665057.617, 178...",4
473797,1,0,1,19.660000,0.0,9,3,4,2,2,...,0,6,22.9,6,0.024888,0.001354,6432,6433,"LINESTRING (178804.17200000002 665057.617, 178...",4
473798,2,0,1,19.660000,0.0,9,3,4,2,2,...,0,6,22.9,6,0.024888,0.001354,6432,6433,"LINESTRING (178804.17200000002 665057.617, 178...",4


In [6]:
# Model without PCA
my_pipeline = Pipeline([
    ('classifier', DecisionTreeRegressor())
])
cv = ShuffleSplit(n_splits=5, test_size=0.3)
param_grid ={}

# Divide the data to training and test
X_train, X_test, y_train, y_test = train_test_split(x_tree, y_tree, test_size=0.2, random_state=0)
# Find the optimal parameters and cross validation
search =GridSearchCV(my_pipeline, param_grid=param_grid,cv=cv,n_jobs=-1)
search.fit(X_train, y_train)
# model accuracy and performance
y_pred =search.best_estimator_.predict(X_test)

# Compute the R2 score for each label
r2_scores = r2_score(y_test, y_pred, multioutput='raw_values')
print('R2 scores:', r2_scores)

# Compute the mean squared error for each label
mse_scores = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print('Mean squared error:', mse_scores)

# Compute the mean absolute error for each label
mae_scores = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Mean absolute error:', mae_scores)

# Compute the mean absolute error for each label
var_scores = explained_variance_score(y_test, y_pred, multioutput='raw_values')
print('Explained variance score:', var_scores)
med_scores = median_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Median absolute error:', med_scores)


from pandas import DataFrame
stat_df  = DataFrame(data={'y_test':y_test,'y_pred':y_pred})
for group in stat_df.groupby('y_test'):
    data_group = group[1]
    print(f'group {group[0]}, samples: {len(data_group)}, MSE:{mean_squared_error(data_group["y_test"],data_group["y_pred"])}')

# Get the best model from the grid search
best_model = search.best_estimator_


# Get the depth of the decision tree in the best model
regressor = best_model.named_steps['classifier']
tree_depth = regressor.get_depth()
print(f'{tree_depth}')


R2 scores: [0.52404724]
Mean squared error: [0.98479961]
Mean absolute error: [0.64848389]
Explained variance score: [0.52408895]
Median absolute error: [0.5]
group 0, samples: 22144, MSE:1.0980739703757225
group 1, samples: 18391, MSE:0.9978642935252144
group 2, samples: 18969, MSE:1.1075102067115352
group 3, samples: 17011, MSE:1.1369718286860135
group 4, samples: 18245, MSE:0.5646889558783228
61


Get specific path in the code based on two condition : 1) impurity< 1 2) samples>1000

In [7]:
# Create a list so that each node has its parent id
left_children = regressor.tree_.children_left
right_children = regressor.tree_.children_right
parent_nodes = [None] * regressor.tree_.node_count

for i in range(regressor.tree_.node_count):
    if left_children[i] != -1:
        parent_nodes[left_children[i]] = i
    if right_children[i] != -1:
        parent_nodes[right_children[i]] = i

In [40]:
right_children

array([179000,  50305,  21306, ..., 349480,     -1,     -1], dtype=int64)

In [15]:


regressor = best_model.named_steps['classifier']
# Get paths to nodes with at least 1000 samples and square error less than 1
rel_nodes_list = []
node_count = regressor.tree_.node_count
for node_id in range(node_count):
    if regressor.tree_.n_node_samples[node_id] >= 1250 and regressor.tree_.impurity[node_id] < 0.75:
        rel_nodes_list.append(node_id)

rel_nodes_list

[38322, 38376, 38377, 38439, 38440, 38441, 44153, 44154, 146787, 171941]

In [16]:
leaf_id = regressor.apply(X_train)
def find_sample(x):
    """
    the code search for the leaf under the current node and then return one of its sample
    :param x:
    :return:
    """
    y= regressor.tree_.children_right[x]
    if y==-1 & regressor.tree_.children_left[x]==-1:
        return X_train[leaf_id == x][0]
    else:
        return find_sample(y)

In [17]:
paths_nodes = []
for node_id in rel_nodes_list:
    # get all the nodes from for a certain path
    parents_node=[]
    i = regressor.tree_.n_node_samples[node_id]
    while i!=0:
        parents_node.append(parent_nodes[i])
        i=parent_nodes[i]
    parents_node.reverse()
    # find one representative sample
    b= find_sample(node_id)

    path_node = []
    j=0
    for i in parents_node:
        feat_i = regressor.tree_.feature[i]
        feat= data_feature.columns[feat_i]
        threshold= regressor.tree_.threshold[i]
        if b[feat_i] <= threshold:
            threshold_sign = "<="
        else:
            threshold_sign = ">"
        path_node.append(f'{j}:{feat} {threshold_sign} {threshold:.2f}')
        j+=1
    path_node.append(f'value:{regressor.tree_.value[node_id][0][0]:.2f}, samples:{regressor.tree_.n_node_samples[node_id]}, impurity:{regressor.tree_.impurity[node_id]:.2f}')
    paths_nodes.append(path_node)
paths_nodes
    # print('\n'.join(path_node))

[['0:day <= 0.50',
  '1:closeness <= 0.02',
  '2:synagogues > 0.50',
  '3:educationa <= 7.50',
  '4:closeness > 0.02',
  '5:lighting > 0.04',
  '6:lighting > 0.00',
  '7:parks > 0.50',
  '8:trees <= 0.02',
  '9:closeness > 0.00',
  '10:density > 10.95',
  '11:closeness > 0.01',
  '12:buildings > 10.63',
  '13:parks > 16.50',
  '14:closeness > 0.01',
  '15:buildings <= 12.31',
  '16:Sport_faci > 2.00',
  '17:day part > 2.50',
  '18:day part > 1.50',
  '19:day part > 0.50',
  '20:Playground > 3.00',
  '21:season > 2.50',
  'value:3.48, samples:1627, impurity:0.72'],
 ['0:day <= 0.50',
  '1:closeness <= 0.02',
  '2:synagogues > 0.50',
  '3:educationa <= 7.50',
  '4:closeness > 0.02',
  '5:lighting > 0.04',
  '6:lighting > 0.00',
  '7:parks > 0.50',
  '8:trees <= 0.02',
  '9:closeness > 0.00',
  '10:density > 10.95',
  '11:closeness > 0.01',
  '12:buildings > 10.63',
  '13:parks > 16.50',
  '14:closeness > 0.01',
  '15:betweennes <= 0.00',
  '16:season > 2.50',
  '17:day part > 1.50',
  'v

In [48]:
parents_node

[0, 1, 2, 3, 4, 5, 6, 7, 527, 1595, 1596, 1597, 1643, 1644, 1645, 1653]

In [49]:


print(b)

[3.00000000e+00 3.00000000e+00 0.00000000e+00 1.06011111e+01
 0.00000000e+00 6.00000000e+00 0.00000000e+00 1.00000000e+00
 3.00000000e+00 3.00000000e+00 3.00000000e+00 0.00000000e+00
 3.15088934e-02 6.30177868e-02 0.00000000e+00 1.60000000e+01
 1.25000000e+01 1.69477751e-02 3.82368131e-04]


In [50]:
paths_nodes = []
paths_nodes.append('\n'.join(path_node))
paths_nodes

['0:day <= 0.50\n1:closeness <= 0.02\n2:synagogues <= 0.50\n3:educationa <= 7.50\n4:closeness > 0.02\n5:lighting > 0.04\n6:lighting > 0.00\n7:buildings > 9.04\n8:density <= 10.95\n9:trees <= 0.00\n10:buildings <= 35.18\n11:parks > 10.00\n12:day part > 2.50\n13:density > 1.15\n14:season > 2.50\nvalue:3.40, samples:1345, impurity:0.78']

In [46]:
paths_nodes = []
path_node = []
j=0
for i in parents_node:
    feat_i = regressor.tree_.feature[i]
    feat= data_feature.columns[feat_i]
    threshold= regressor.tree_.threshold[i]
    if b[feat_i] <= threshold:
        threshold_sign = "<="
    else:
        threshold_sign = ">"
    path_node.append(f'{j}:{feat} {threshold_sign} {threshold:.2f}')
    j+=1
path_node.append(f'value:{regressor.tree_.value[node_id][0][0]:.2f}, samples:{regressor.tree_.n_node_samples[node_id]}, impurity:{regressor.tree_.impurity[node_id]:.2f}')
print('\n'.join(path_node))

0:day <= 0.50
1:closeness <= 0.02
2:synagogues <= 0.50
3:educationa <= 7.50
4:closeness > 0.02
5:lighting > 0.04
6:lighting > 0.00
7:buildings > 9.04
8:density <= 10.95
9:trees <= 0.00
10:buildings <= 35.18
11:parks > 10.00
12:day part > 2.50
13:density > 1.15
14:season > 2.50
value:3.40, samples:1345, impurity:0.78


In [42]:
regressor.tree_.value[node_id][0][0]

3.398513011152416

In [10]:
regressor.tree_.feature[0]

2

In [43]:
regressor.tree_.mse[0] < 1

AttributeError: 'sklearn.tree._tree.Tree' object has no attribute 'mse'

In [45]:

regressor.tree_.impurity

array([2.07320921, 2.02234139, 1.89278135, ..., 0.22222222, 0.        ,
       0.        ])

In [4]:
DataFrame(data={'feature_importance':best_model.named_steps['classifier'].feature_importances_.round(3)*100,'features_name':data_feature.columns}).sort_values('feature_importance',ascending=0)

,feature_importance,features_name
17,11.4,closeness
2,10.9,day
18,8.9,betweennes
3,8.8,buildings
1,8.4,season
12,8.2,lighting
0,6.2,day part
15,5.8,parks
13,5.1,trees
16,4.8,density


In [17]:
# Tree for seven levels
from sklearn import tree
import matplotlib.pyplot as plt
fig, axes2 = plt.subplots(nrows=1, ncols=1, figsize=(20, 20), dpi=500)
tree.plot_tree(best_model.named_steps['classifier'],
               feature_names= data_feature.columns,
               class_names=list(ml_df[ped_level].sort_values().unique().astype(str))
               )
fig.savefig('output/tree_7.pdf')

In [ ]:
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import _tree

# Generate sample data
X, y = make_regression(n_samples=10000, n_features=10, n_informative=5, random_state=42)

# Train decision tree regressor
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X, y)

# Get paths to nodes with at least 1000 samples and square error less than 1
path_list = []
node_count = len(regressor.tree_.n_node_samples)
leaf_id = regressor.apply(X)
for node_id in range(node_count):
    if regressor.tree_.n_node_samples[node_id] >= 1000 and regressor.tree_.mse[node_id] < 1:
        path = []
        while node_id != _tree.TREE_LEAF:
            feature = regressor.tree_.feature[node_id]
            threshold = regressor.tree_.threshold[node_id]
            if threshold != -2:
                if X[leaf_id == node_id, feature] <= threshold:
                    threshold_sign = "<="
                else:
                    threshold_sign = ">"
                path.append(f"({data_feature.columns[regressor.tree_.feature[]]} {threshold_sign} {threshold:.2f})")
            node_id = regressor.tree_.children_left[node_id] if X[leaf_id == node_id, feature] <= threshold else regressor.tree_.children_right[node_id]
        path_list.append(path)

# Print the paths to nodes with at least 1000 samples and square error less than 1
for path in path_list:
    print(' '.join(path))

In [9]:
data_feature.columns[19]

IndexError: index 19 is out of bounds for axis 0 with size 19

In [7]:
# Get the best model from the grid search
best_model = search.best_estimator_

# Get the depth of the decision tree in the best model
tree_depth = best_model.named_steps['classifier'].get_depth()
tree_depth

53

In [30]:
from pandas import DataFrame
stat_df  = DataFrame(data={'y_test':y_test,'y_pred':y_pred})
stat_df

,y_test,y_pred
0,0,0.0
1,2,0.0
2,2,1.0
3,2,3.0
4,0,0.0
...,...,...
94755,3,4.0
94756,2,1.0
94757,3,1.0
94758,4,4.0


In [35]:
for group in stat_df.groupby('y_test'):
    data_group = group[1]
    print(f'group {group[0]}, samples: {len(data_group)}, MSE:{mean_squared_error(data_group["y_test"],data_group["y_pred"])}')



group 0, samples: 22144, MSE:1.110884318413059
group 1, samples: 18391, MSE:0.9843300769095995
group 2, samples: 18969, MSE:1.1011957960649243
group 3, samples: 17011, MSE:1.1457972619024293
group 4, samples: 18245, MSE:0.5744498492737736


In [22]:
# Model without tuning with PCA

my_pipeline = Pipeline([
    ('pca', PCA()),
    ('classifier', DecisionTreeRegressor())
])
cv = ShuffleSplit(n_splits=5, test_size=0.3)


param_grid ={}

# Divide the data to training and test
X_train, X_test, y_train, y_test = train_test_split(x_tree, y_tree, test_size=0.2, random_state=0)
# Find the optimal parameters and cross validation
search =GridSearchCV(my_pipeline, param_grid=param_grid,cv=cv,n_jobs=-1)
search.fit(X_train, y_train)
# model accuracy and performance
y_pred =search.best_estimator_.predict(X_test)

# Compute the R2 score for each label
r2_scores = r2_score(y_test, y_pred, multioutput='raw_values')
print('R2 scores:', r2_scores)

# Compute the mean squared error for each label
mse_scores = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print('Mean squared error:', mse_scores)

# Compute the mean absolute error for each label
mae_scores = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Mean absolute error:', mae_scores)

# Compute the mean absolute error for each label
var_scores = explained_variance_score(y_test, y_pred, multioutput='raw_values')
print('Explained variance score:', var_scores)
med_scores = median_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Median absolute error:', med_scores)

R2 scores: [0.51930274]
Mean squared error: [0.99461652]
Mean absolute error: [0.65353349]
Explained variance score: [0.51931166]
Median absolute error: [0.5]


In [25]:
# Model without centrality
data_feature_1 = data_feature.drop(columns=['closeness','betweennes'])
x_tree = data_feature_1.to_numpy()
y_tree = ml_df[ped_level].to_numpy()

my_pipeline = Pipeline([
    ('classifier', DecisionTreeRegressor())
])
cv = ShuffleSplit(n_splits=5, test_size=0.3)

param_grid ={}

# Divide the data to training and test
X_train, X_test, y_train, y_test = train_test_split(x_tree, y_tree, test_size=0.2, random_state=0)
# Find the optimal parameters and cross validation
search =GridSearchCV(my_pipeline, param_grid=param_grid,cv=cv,n_jobs=-1)
search.fit(X_train, y_train)
# model accuracy and performance
y_pred =search.best_estimator_.predict(X_test)

# Compute the R2 score for each label
r2_scores = r2_score(y_test, y_pred, multioutput='raw_values')
print('R2 scores:', r2_scores)

# Compute the mean squared error for each label
mse_scores = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print('Mean squared error:', mse_scores)

# Compute the mean absolute error for each label
mae_scores = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Mean absolute error:', mae_scores)

# Compute the mean absolute error for each label
var_scores = explained_variance_score(y_test, y_pred, multioutput='raw_values')
print('Explained variance score:', var_scores)
med_scores = median_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Median absolute error:', med_scores)

R2 scores: [0.52205201]
Mean squared error: [0.98892797]
Mean absolute error: [0.65017176]
Explained variance score: [0.52207558]
Median absolute error: [0.5]


In [3]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import RandomizedSearchCV

my_pipeline = Pipeline([
    ('classifier', DecisionTreeRegressor())
])
cv = ShuffleSplit(n_splits=5, test_size=0.3,random_state=0)

param_grid = {'classifier__criterion': ["squared_error", "absolute_error"],"classifier__max_depth": range(10,45,5),'classifier__min_samples_split':range(2,20,2),'classifier__max_features':['auto','sqrt','log2',None]}


# Divide the data to training and test
X_train, X_test, y_train, y_test = train_test_split(x_tree, y_tree, test_size=0.2, random_state=0)
# Find the optimal parameters and cross validation
search =RandomizedSearchCV(my_pipeline, param_distributions=param_grid,cv=cv,n_jobs=-1,verbose=2)
search.fit(X_train, y_train)
# model accuracy and performance
y_pred =search.best_estimator_.predict(X_test)

# Compute the R2 score for each label
r2_scores = r2_score(y_test, y_pred, multioutput='raw_values')
print('R2 scores:', r2_scores)

# Compute the mean squared error for each label
mse_scores = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print('Mean squared error:', mse_scores)

# Compute the mean absolute error for each label
mae_scores = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Mean absolute error:', mae_scores)

# Compute the mean absolute error for each label
var_scores = explained_variance_score(y_test, y_pred, multioutput='raw_values')
print('Explained variance score:', var_scores)
med_scores = median_absolute_error(y_test, y_pred, multioutput='raw_values')
print('Median absolute error:', med_scores)

# feature importance
# best_params = list(search.best_params_.values())
# dt = DecisionTreeClassifier(random_state=0,criterion= best_params[0], max_features=best_params[1], splitter=best_params[2]).fit(x_tree, y_tree)
# new_df = DataFrame(data={'feature_importance':dt.feature_importances_.round(3)*100,'features_name':data_feature.columns}).sort_values('feature_importance',ascending=0)
# print(new_df)

OSError: [Errno 22] Invalid argument